In [53]:
import os
envir = os.environ

In [54]:
envir["GOOGLE_APPLICATION_CREDENTIALS"]='C:\\Users\\Safa Eladib\\Downloads\\My Project 68340-324c0b65d5ac.json'

In [55]:
from google.cloud import bigquery

In [56]:
client = bigquery.Client(project='PROJECT_ID')

In [57]:
hn_dataset_ref = client.dataset('openaq', project='bigquery-public-data')

In [58]:
hn_dset = client.get_dataset(hn_dataset_ref)

In [59]:
type(hn_dset)

google.cloud.bigquery.dataset.Dataset

In [60]:
[x.table_id for x in client.list_tables(hn_dset)]

['global_air_quality']

In [27]:
hn_full = client.get_table(hn_dset.table('global_air_quality'))

In [28]:
schema = hn_full.schema
schema

[SchemaField('location', 'STRING', 'NULLABLE', 'Location where data was measured', ()),
 SchemaField('city', 'STRING', 'NULLABLE', 'City containing location', ()),
 SchemaField('country', 'STRING', 'NULLABLE', 'Country containing measurement in 2 letter ISO code', ()),
 SchemaField('pollutant', 'STRING', 'NULLABLE', 'Name of the Pollutant being measured. Allowed values: PM25, PM10, SO2, NO2, O3, CO, BC', ()),
 SchemaField('value', 'FLOAT', 'NULLABLE', 'Latest measured value for the pollutant', ()),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'The datetime at which the pollutant was measured, in ISO 8601 format', ()),
 SchemaField('unit', 'STRING', 'NULLABLE', 'The unit the value was measured in coded by UCUM Code', ()),
 SchemaField('source_name', 'STRING', 'NULLABLE', 'Name of the source of the data', ()),
 SchemaField('latitude', 'FLOAT', 'NULLABLE', 'Latitude in decimal degrees. Precision >3 decimal points.', ()),
 SchemaField('longitude', 'FLOAT', 'NULLABLE', 'Longitude in d

In [29]:
schema_subset = [col for col in hn_full.schema if col.name in ('location', 'city', 'country','pollutant','value','timestamp','unit','source_name','latitude','longitude','averaged_over_in_hours')]
results = [x for x in client.list_rows(hn_full, start_index=1, selected_fields=schema_subset)]

In [30]:
print(len(results))

21212


In [31]:
air_full = pd.DataFrame()
j = range(0,len(results)-1)
for i,j in zip(results,j):
     air_full = air_full.append(pd.DataFrame(dict(i),index=[j]))

In [32]:
air_full

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.609220,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.609220,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.609220,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.609220,0.25
4,"BWSSB Kadabesanahalli, Bengaluru - KSPCB",Bengaluru,IN,co,840.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.938906,77.697270,0.25
...,...,...,...,...,...,...,...,...,...,...,...
21206,ES1980A,Santa Cruz de Tenerife,ES,o3,36.00,2020-05-16 08:00:00+00:00,µg/m³,EEA Spain,28.685984,-17.764587,1
21207,ES1980A,Santa Cruz de Tenerife,ES,pm10,32.00,2020-05-16 08:00:00+00:00,µg/m³,EEA Spain,28.685984,-17.764587,1
21208,ES1980A,Santa Cruz de Tenerife,ES,pm25,12.00,2020-05-16 08:00:00+00:00,µg/m³,EEA Spain,28.685984,-17.764587,1
21209,ES1980A,Santa Cruz de Tenerife,ES,so2,12.00,2020-05-16 08:00:00+00:00,µg/m³,EEA Spain,28.685984,-17.764587,1


In [33]:
air_full.to_csv('data/air_quality_full.csv')